In [10]:
from ba137_hyperfine_hamiltonian_solver import *
import numpy as np

In [5]:
sensitivity_matrix = generate_magnetic_field_sensitivity(4.209)

In [7]:
transition_strength = TransitionStrength(4.209)

In [12]:
print(np.round(transition_strength,5))

[[0.08204 0.14652 0.25298 0.      0.05633 0.055   0.05188 0.03407]
 [0.13767 0.19292 0.1732  0.07524 0.02457 0.0096  0.0336  0.05602]
 [0.2251  0.16416 0.11572 0.05299 0.0598  0.04697 0.03553 0.     ]
 [0.13254 0.23231 0.      0.      0.      0.11031 0.10614 0.08194]
 [0.1854  0.12778 0.14289 0.      0.12732 0.02754 0.06124 0.0981 ]
 [0.18197 0.03656 0.16893 0.09731 0.0635  0.09192 0.02583 0.09323]
 [0.17408 0.07232 0.19431 0.11626 0.02819 0.0575  0.10701 0.     ]
 [0.      0.21318 0.15734 0.11049 0.10561 0.08141 0.      0.     ]
 [0.16792 0.      0.      0.      0.      0.      0.28134 0.00907]
 [0.12545 0.07642 0.      0.      0.      0.22882 0.15491 0.0415 ]
 [0.10837 0.0911  0.02714 0.      0.15172 0.21347 0.06967 0.04458]
 [0.07403 0.10948 0.04513 0.07631 0.20083 0.17404 0.00305 0.03589]
 [0.04927 0.1008  0.07332 0.13668 0.23514 0.07723 0.03898 0.     ]
 [0.      0.09248 0.09337 0.22409 0.17624 0.01058 0.      0.     ]
 [0.      0.      0.13258 0.26801 0.06339 0.      0.      0.  

In [15]:
transition_frequencies = generate_frequencies_at(b = 4.209, f0 = 545.116618)

In [16]:
print(transition_frequencies)

[[         nan 457.60832477 460.55983265 463.50917745 466.45636391]
 [466.80682415 469.76049989 472.71200778 475.66135257 478.60853903]
 [479.31367092 482.26734666 485.21885455 488.16819934          nan]
 [         nan          nan 531.7653726  534.71471739 537.66190385]
 [         nan 535.55631312 538.50782101 541.4571658  544.40435226]
 [539.21143437 542.16511011 545.116618   548.06596279 551.01314925]
 [545.6274373  548.58111305 551.53262093 554.48196573          nan]
 [551.75353106 554.7072068  557.65871469          nan          nan]
 [         nan          nan          nan 593.43749932 596.38468578]
 [         nan          nan 594.48678331 597.43612811 600.38331456]
 [         nan 595.89768526 598.84919315 601.79853794 604.7457244 ]
 [597.55279776 600.5064735  603.45798139 606.40732618 609.35451264]
 [602.38413221 605.33780796 608.28931585 611.23866064          nan]
 [607.47809365 610.4317694  613.38327729          nan          nan]
 [613.00038904 615.95406479          nan        

In [9]:
import numpy as np
from scipy.linalg import expm

def build_coupling_hamiltonian(coupling, phase, rabi_freqs_kHz, mag_sen_MHz_per_G, delta_B_G):
    H = np.zeros((3, 3), dtype=complex)
    i, j = coupling
    key = (i, j)
    if key not in rabi_freqs_kHz and (j, i) in rabi_freqs_kHz:
        key = (j, i)
        i, j = j, i
    Omega = 2 * np.pi * rabi_freqs_kHz[key] * 1e3
    if key not in mag_sen_MHz_per_G and (j, i) in mag_sen_MHz_per_G:
        key = (j, i)
    Delta = 2 * np.pi * mag_sen_MHz_per_G[key] * 1e6 * delta_B_G
    H[i, i] -= Delta / 2.0
    H[j, j] += Delta / 2.0
    hij = 0.5 * Omega * np.exp(1j * phase)
    H[i, j] += hij
    H[j, i] += np.conjugate(hij)
    return H, Omega

def propagate(H, psi, duration, samples_per_pulse):
    dt = duration / samples_per_pulse
    U_dt = expm(-1j * H * dt)
    psi_out = psi
    times = []
    states = []
    for _ in range(samples_per_pulse):
        psi_out = U_dt @ psi_out
        times.append(dt)
        states.append(psi_out.copy())
    return np.array(times), np.vstack(states)

def simulate_three_level(pulse_train,
                         fractions,
                         phases,
                         rabi_freqs_kHz,
                         mag_sen_MHz_per_G,
                         transition_freqs_MHz=None,
                         B_field_G=0.0,
                         B_ref_G=0.0,
                         psi0=None,
                         wait_time=0.0,
                         samples_per_pulse=1):
    if psi0 is None:
        psi0 = np.array([1.0 + 0j, 0.0 + 0j, 0.0 + 0j])
    psi = psi0.astype(complex)
    delta_B_G = B_field_G - B_ref_G
    t_curr = 0.0
    times = [0.0]
    states = [psi.copy()]
    for step, frac, phase in zip(pulse_train, fractions, phases):
        if step == "Wait":
            if wait_time <= 0:
                continue
            H_step = np.zeros((3, 3), dtype=complex)
            duration = wait_time
        else:
            H_step, Omega = build_coupling_hamiltonian(step, phase, rabi_freqs_kHz, mag_sen_MHz_per_G, delta_B_G)
            if isinstance(frac, (int, float, np.floating)):
                t_pi = np.pi / Omega
                duration = frac * t_pi
            else:
                raise ValueError("Non-numeric fraction for a pulse step.")
        local_times, local_states = propagate(H_step, psi, duration, samples_per_pulse)
        for dt, st in zip(local_times, local_states):
            t_curr += dt
            times.append(t_curr)
            states.append(st)
        psi = states[-1]
    return np.array(times), np.vstack(states)

if __name__ == "__main__":
    pulse_train = [(0, 1), (0, 2), "Wait", (0, 2), (0, 1)]
    fractions = [0.5, 1.0, "Wait", 1.0, 0.5]
    phases = [0.0, 1.0, "Wait", 0.0, 1.0]
    rabi_freqs_kHz = {
        (0, 1): 10.0,
        (0, 2): 10.0,
    }
    mag_sen_MHz_per_G = {
        (0, 1): 0.5,
        (0, 2): -0.2,
    }
    transition_freqs_MHz = {
        (0, 1): 500.0,
        (0, 2): 500.0,
    }
    B_field_G = 1.0
    B_ref_G = 1
    psi0 = np.array([1.0 + 0j, 0.0 + 0j, 0.0 + 0j])
    times, states = simulate_three_level(
        pulse_train=pulse_train,
        fractions=fractions,
        phases=phases,
        rabi_freqs_kHz=rabi_freqs_kHz,
        mag_sen_MHz_per_G=mag_sen_MHz_per_G,
        transition_freqs_MHz=transition_freqs_MHz,
        B_field_G=B_field_G,
        B_ref_G=B_ref_G,
        psi0=psi0,
        wait_time=50e-6,
        samples_per_pulse=50,
    )
    populations = np.abs(states) ** 2
    print("Final populations:", populations[-1])


Final populations: [2.91926582e-01 7.08073418e-01 3.96771661e-32]
